In [340]:
import pandas as pd

In [341]:
books = pd.read_csv("bannedBooks.csv", index_col=0)
lea = pd.read_csv("lea_characteristics.csv", low_memory=False)

In [342]:
books["STATE"] = books["State"].str.upper()

In [343]:
# preprocessing district names

# lowercase
books['district_name'] = books['District'].str.lower()
lea['district_name'] = lea['LEA_NAME'].str.lower()

# lea: isd = independent school district, sd = school district


# remove common words, so we don't inflate similarity score
# phrases = ["independent school district", "isd", "school district", "sd", "county public schools", "co pblc schs", "public schools", "county"]
common_words = ["city school district", "schools", "school", "district", "public", "county", 
                "independent", "cisd", "isd", " sd", " co ", "pblc", "schs", "high", "of", 
                "community", "comm", "area", "the", "department", "consolidated", "  no",
                "r-iii", "r-vii", "r-ix", "r-iv", "r-1", "r-i", 
                "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "#", "-", "."]

for word in common_words: 
    books['district_name'] = books['district_name'].str.replace(word, " ").str.strip()
    lea['district_name'] = lea['district_name'].str.replace(word, " ").str.strip()
    


/var/folders/94/222wfkls7wgcc_d0bpzq1nbh0000gn/T/ipykernel_68122/3429104261.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  books['district_name'] = books['district_name'].str.replace(word, " ").str.strip()
/var/folders/94/222wfkls7wgcc_d0bpzq1nbh0000gn/T/ipykernel_68122/3429104261.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  lea['district_name'] = lea['district_name'].str.replace(word, " ").str.strip()


In [385]:
pairs = {}
for district in pd.unique(books["district_name"]): 
    pen_names = books[books["district_name"]==district][["STATE", "District"]]
    lea_names = lea[lea["district_name"]==district][["LEA_STATE_NAME", "LEA_NAME"]]
    
    for pen_name in pd.unique(pen_names["District"]): 
        state = pen_names[pen_names["District"] == pen_name]["STATE"].values[0]
        lea_name = lea_names[lea_names["LEA_STATE_NAME"] == state]["LEA_NAME"].values
        if len(lea_name) > 0: 
            lea_name = lea_name[0]
            pairs[pen_name] = lea_name

In [347]:
outliers_pen_to_lea = {
    "Medford School District 549C": "Medford SD 549C", 
    "Virginia Beach City Public Schools": "VA BEACH CITY PBLC SCHS",
    "Harlem School District 122": "Harlem UD 122", 
    "Regional School Unit No. 56": "RSU 56", 
    "ROWVA CUSD 208": "R O W V A CUSD 208", 
    "Cherry Creek School District": "Cherry Creek School District No. 5 in the County of Arapaho", 
    "Raytown Quality Schools": "RAYTOWN C-2", 
    "Williston Basin School District #7": "WILLISTON 1", 
    "Lexington-Richland School District 5": "School District 5 of Lexington and Richland Counties", 
    "Madison County Schools": "MADISON CO SCHOOL DIST", 
}

matching_district_names = pairs | outliers_pen_to_lea

In [303]:
# for word in not_yet:
#     print(word)
#     print(books[books['district_name'] == word][["STATE", "District", "district_name"]])
#     print(lea[lea['district_name'].str.contains(word)][["LEA_STATE_NAME", "LEA_NAME", "district_name"]])

medford        c
      STATE                      District     district_name
147  OREGON  Medford School District 549C  medford        c
Empty DataFrame
Columns: [LEA_STATE_NAME, LEA_NAME, district_name]
Index: []
west ada
      STATE                  District district_name
492   IDAHO  West Ada School District      west ada
1228  IDAHO  West Ada School District      west ada
Empty DataFrame
Columns: [LEA_STATE_NAME, LEA_NAME, district_name]
Index: []
virginia beach city
         STATE                            District        district_name
638   VIRGINIA  Virginia Beach City Public Schools  virginia beach city
722   VIRGINIA  Virginia Beach City Public Schools  virginia beach city
956   VIRGINIA  Virginia Beach City Public Schools  virginia beach city
1261  VIRGINIA  Virginia Beach City Public Schools  virginia beach city
1293  VIRGINIA  Virginia Beach City Public Schools  virginia beach city
1650  VIRGINIA  Virginia Beach City Public Schools  virginia beach city
1734  VIRGINIA  Virg

In [310]:

# print(lea[lea['LEA_STATE_NAME'] == "MISSISSIPPI"]
#       [lea['district_name'].str.contains("madison")]
#       [["LEA_NAME", "LEA_STATE_NAME", "district_name"]]
#      )

                    LEA_NAME LEA_STATE_NAME   district_name
8325  MADISON CO SCHOOL DIST    MISSISSIPPI  madison   dist


/var/folders/94/222wfkls7wgcc_d0bpzq1nbh0000gn/T/ipykernel_68122/510876270.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(lea[lea['LEA_STATE_NAME'] == "MISSISSIPPI"]


In [350]:
matching_district_names

{'Indian River County School District': 'INDIAN RIVER',
 'Central York School District': 'Central York SD',
 'Marlboro Central School District': 'MARLBORO CENTRAL SCHOOL DISTRICT',
 'Fredericksburg Independent School District': 'FREDERICKSBURG ISD',
 'New Kent County Public Schools': 'NEW KENT CO PBLC SCHS',
 'Spotsylvania County Public Schools': 'SPOTSYLVANIA CO PBLC SCHS',
 'Duval County Public Schools': 'DUVAL',
 'Katy Independent School District': 'KATY ISD',
 'Katy ISD': 'KATY ISD',
 'Hudsonville Public Schools': 'Hudsonville Public School District',
 'Collierville Schools': 'Collierville',
 'North East Independent School District': 'NORTH EAST ISD',
 'Madison County Public Schools': 'MADISON CO PBLC SCHS',
 'School District of Waukesha': 'Waukesha School District',
 'Nampa School District': 'NAMPA SCHOOL DISTRICT',
 'Granbury Independent School District': 'GRANBURY ISD',
 'Klein Independent School District': 'KLEIN ISD',
 'Jackson County School District': 'JACKSON',
 'Lee County 

In [357]:
books['matched_name'] = [matching_district_names.get(d, None) for d in books['District']]

In [393]:
merged = books.dropna(subset=['matched_name']).merge(lea, how='left', left_on=['matched_name', 'STATE'], right_on=['LEA_NAME', 'LEA_STATE_NAME'])

In [396]:
merged.columns

Index(['Author', 'Title', 'Type of Ban', 'Secondary Author(s)',
       'Illustrator(s)', 'Translator(s)', 'State', 'District',
       'Date of Challenge/Removal', 'Origin of Challenge', 'Series Name',
       'Ban Status', 'STATE', 'district_name_x', 'matched_name', 'LEA_STATE',
       'LEA_STATE_NAME', 'LEAID', 'LEA_NAME', 'LEA_ADDRESS', 'LEA_CITY',
       'LEA_ZIP', 'CJJ', 'LEA_ENR', 'LEA_ENR_NONLEAFAC', 'LEA_SCHOOLS',
       'LEA_CRCOORD_SEX_IND', 'LEA_CRCOORD_RAC_IND', 'LEA_CRCOORD_DIS_IND',
       'LEA_CRCOORD_SEX_FN', 'LEA_CRCOORD_SEX_LN', 'LEA_CRCOORD_SEX_PH',
       'LEA_CRCOORD_SEX_EM', 'LEA_CRCOORD_RAC_FN', 'LEA_CRCOORD_RAC_LN',
       'LEA_CRCOORD_RAC_PH', 'LEA_CRCOORD_RAC_EM', 'LEA_CRCOORD_DIS_FN',
       'LEA_CRCOORD_DIS_LN', 'LEA_CRCOORD_DIS_PH', 'LEA_CRCOORD_DIS_EM',
       'LEA_DESEGPLAN', 'LEA_HBPOLICY_IND', 'LEA_HBPOLICYURL_IND',
       'LEA_HBPOLICY_URL', 'LEA_ECE_IND', 'LEA_ECE_NONIDEA', 'LEA_PS_IND',
       'LEA_PS_FULLDAYFREE', 'LEA_PS_FULLDAYCOST', 'LEA_PS_PARTDAY

In [397]:
cleaned_df = merged[['LEAID', 'LEA_NAME', 'LEA_STATE_NAME', 'Title', 'Author', 'Type of Ban', 'Date of Challenge/Removal', 'Origin of Challenge', 'Series Name', 'Ban Status']]

In [399]:
cleaned_df.to_csv('pen_to_lea.csv')